<a href="https://colab.research.google.com/github/bltsezer/DataSet_Uuygulama/blob/main/_01__fashionMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Burada gerekli paketleri import edip,tensorflow versiyonu yazdiralım

In [ ]:
import tensorflow as tf
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

Burada 10 kategoriye ayrılmış, 70.000 adet gri formatlı resimler içeren Fashion MNIST veri seti kullacacağız. Görüntüler,düşük çözünürlüğe (28 x 28 piksel) sahiptir.Sonrasında Verisetimizi 'load_data()' fonksiyonu ile Eğitim ve Test diye ikiye ayırıyoruz.Eğitim ve test verisinde,  görüntü ve  sayısal etiket olarak ayırıyoruz

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

veri setini inceleyelim.'shape' sözcügü boyutu hakkında bilgi verir

gri format  (adet,genislik,yükseklik)

RGB format  (adet,genislik,yükseklik,kanalSayısı=3)
!!!: kanal sayısı varsayılan olarak 3 yazılır(Red,Green,Blue)

In [ ]:
print("egitim verisinin boyutu: ",train_images.shape)
print("egitim verisinindeki görüntü sayısı: ",train_images.shape[0])
print("egitim verisinindeki etiket boyutu: ",train_labels.shape)
print("egitim verisinindeki etiket sayısı: ",train_labels.shape[0])
print("\ntest verisinin boyutu: ",test_images.shape)
print("test verisinindeki görüntü sayısı: ",test_images.shape[0])
print("test verisinindeki etiket boyutu: ",test_labels.shape)
print("test verisinindeki etiket sayısı: ",test_labels.shape[0])

Resimleri metinlendirmek için bir liste olusturup,içerisine kategoriye göre sınıf isimlerini yazalım.

In [ ]:
class_names = ['T-shirt', 'Pantolon', 'Kazak', 'Elbise', 'Palto',
               'Terlik', 'Gömlek', 'Spor', 'Çanta', 'Bot']

Görüntü içerisinde pixel değerlerinin 0-255 arası olduğunu görelim

Örnek olarak ilk resmin pixel ve etiket değerini yazalım

In [ ]:
print("Bilgisayar resmi böyle görür \n")
print(train_images[0],"\n")

Biz resme bakınca böyle görüyoruz.

In [ ]:
plt.figure()
plt.imshow(train_images[0],cmap=plt.cm.binary)
plt.grid(False)
plt.show()

Resmi bir onişleme tabi tutalım.Pixel degerlerini 255.0 ile bölüp 0 ile 1 arasındaki değerlere(dtype=float64) atayalım.

Farklı standardization için buraya https://scikit-learn.org/stable/modules/preprocessing.html tiklayabilirsiniz

In [ ]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler,MaxAbsScaler

train_images_sca = train_images/255.0
test_images_sca = test_images/255.0


#scaler = MinMaxScaler()
#scaler.fit(train_images[10])
#train_images_mms =scaler.transform( train_images[10])
#test_images_mms =scaler.transform(test_images[10])

#scaler2 = StandardScaler()
#scaler2.fit(train_images[10])
#train_images_mms =scaler2.transform( train_images[10])
#test_images_mms =scaler2.transform(test_images[10])

Örnek olarak rastgele 9 adet görüntüyü alalım

In [ ]:
plt.figure(figsize=(10,10))
for i in range(9):
  r=np.random.randint(1000)
  plt.subplot(3,3,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(train_images_sca[r], cmap=plt.cm.binary)
  plt.xlabel(class_names[train_labels[r]])
plt.show()

Görüntüyü tahmin edecek modeli yazalim.

Modelimiz 4 katmanlı,giris olarak 28*28 lik görüntü alır.Çıkış olarak kategoride etiketlenmis sınıfı tahmin oranlarını gösterecektir.

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10)
])

Modeli derlerken varsayılan olarak optiize edici olarak 'adam',metrics olarak 'accuracy' parametreleri verilsin.

https://www.tensorflow.org/api_docs/python/tf/keras/optimizers

https://www.tensorflow.org/api_docs/python/tf/keras/metrics

https://www.tensorflow.org/api_docs/python/tf/keras/losses

adresinden diğer parametreleri deneyebilirsiniz



In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

Modeli eğitirken verilerin tamamı aynı anda eğitime katılmaz.Modelin bazı parçalar halinde eğitimde yer alırlar.İlk kısım eğitilip, modelin başarı durumu test edilir, başarı durumuna  göre geriyeyayılım ile ağırlıklar güncellenir.

 Daha sonra yeni eğitim kümesi ile model tekrar eğitilir ve ağırlıklar tekrar güncellenir. Bu işlem her bir eğitim adımında tekrarlanarak model için en uygun ağırlık değerleri hesaplanmaya çalışılır. Bu adımlara EPOCH diyoruz.

 Epoch degerini 10 atalim ve modeli fit edelim.

In [ ]:
model.fit(train_images_sca, train_labels, epochs=10)

Modelin dogruluk'accuracy' ve kayıp'loss' degerlerini yazdiralim

In [ ]:
train_loss, train_acc = model.evaluate(train_images_sca,  train_labels, verbose=2)
test_loss, test_acc = model.evaluate(test_images_sca,  test_labels, verbose=2)
print('\nTrain accuracy:', train_acc)
print('Train loss:', train_loss)

print('\nTest accuracy:', test_acc)
print('Test loss:', test_loss)

Tahminciyi(prediction yani) olustıralım

In [ ]:
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
predictions = probability_model.predict(test_images_sca)

rastgele 2 tanesini yazdıralım

In [ ]:
import numpy as np
for i in range(0,2):
  rnd=np.random.randint(10)
  print(f"---------{i+1} resim-----------")
  print(rnd,". test degeri:\n",test_images_sca[rnd])
  print(rnd,". test etiketi: ",class_names[test_labels[rnd]])
  print("")
  print(rnd,". tahmin degeri: \n",predictions[rnd])
  print(rnd,". tahmin etiketi: ",class_names[np.argmax(predictions[rnd])])
  print("--------------------------------------\n")

görüntüyü ve tahminleri çizdirecek fonksiyonu yazalim

In [ ]:
def plot_image(i, predictions_array, true_label, img):
  true_label, img = true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  true_label = true_label[i]
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

görüntüleri ve tahminleri inceleyelim

In [ ]:

num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions[i], test_labels, test_images)
  
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions[i], test_labels)
plt.tight_layout()
plt.show()

Model testten gelen görüntüyü nasıl tahmin etmiş

In [ ]:

r=int(input("resim için (0-10000) arası deger gir: "))
max_l=test_images_sca.shape[0]
if r>max_l:
  r=int(input(f"{max_l}den daha kucuk deger gir: "))

img = (np.expand_dims(test_images_sca[r],0))
print(test_labels[r],"= ",class_names[test_labels[r]])
,
predictions_single = probability_model.predict(img)

print(predictions_single)

plot_value_array(1, predictions_single[0], test_labels)
_ = plt.xticks(range(10), class_names, rotation=45)
plt.show()

np.argmax(predictions_single[0])

# Artık uyku vakti